# Hinode-XRT: A Practical Guide to Data Extraction and Visualization

This notebook provides a step-by-step guide on how to download, process, and visualize solar observation data from the Hinode X-Ray Telescope, making it accessible for research and analysis.

*Created by Joy Velasquez. Version 1 - February 29, 2024*

## Introduction

This notebook is designed as a preliminary guide to handle solar observation data specifically from the Hinode X-Ray Telescope (XRT). While this example does not directly utilize `xrtpy` functionalities, it lays the groundwork for handling and understanding XRT data, which can be further processed and analyzed using `xrtpy`.

Here's what we'll cover:
- **Downloading XRT Data**: Learn how to download XRT data for a specific period of interest.
- **Data Filtering**: Use custom functions to filter and process the downloaded data.
- **Visualization and Movie Creation**: Explore methods to visualize the data and compile it into movie format, providing a dynamic view of solar observations.

Whether for research analysis or as a stepping stone to using `xrtpy` for more advanced applications, this guide aims to equip you with the fundamental skills necessary for effective data manipulation and visualization in solar physics. 

## Table of Contents
1. [Importing Necessary Packages](#importing-packages)
2. [Using Fido to Search and Download XRT Data](#using-fido-to-search-and-download-xrt-data)
3. [Inspecting the Downloaded Data](#inspecting-the-downloaded-data)
4. [Exploring Functions to Enhance Data Quality](#exploring-functions-to-enhance-data-quality)
   - 4.1 [Filtering FITS Files](#filtering-fits-files-through-selected-filter)
   - 4.2 [Filtering FITS Files Through Pixel Size](#filtering-fits-files-through-pixel-size)
5. [Navigating FITS Images](#Navigating-FITS-Images)
6. [Removing Unwanted FITS Files](#Removing-unwanted-FITS-Files)

6. [Creating a Movie from XRT FITS Data](#Creating-a-Movie-from-XRT-FITS-Data)

<a id="importing-packages"></a>
## Importing Necessary Packages

The following code cell contains all the necessary Python packages and modules imports to set up our working environment for handling Hinode XRT data.

In [ ]:
import os
import tempfile

import imageio
import matplotlib.pyplot as plt

import sunpy.map
from IPython.display import Video

from sunpy.net import Fido
from sunpy.net import attrs as a


from IPython.display import display
import ipywidgets as widgets

from IPython.display import display, clear_output


# Ensure you have the necessary libraries installed by running the line below
# pip install sunpy matplotlib imageio ipywidgets

<a id="using-fido-to-search-and-download-xrt-data"></a>
## Using Fido to Search and Download XRT Data


[SunPy]:https://sunpy.org/

Fido is a unified interface provided by [SunPy] for querying and downloading data from various solar physics instruments. In this part of the notebook, we will use Fido to search for specific datasets from the Hinode X-Ray Telescope (XRT). We will demonstrate how to define a time range and select the instrument to execute the data search effectively.

We will focus on the Hinode-XRT observation of [Active Region (AR) 13234](https://www.solarmonitor.org/region_pop.php?date=20230223&type=hxrt_flter&region=13234), which was recorded on February 23, 2023, around 18:00 UT, near the solar limb. This region appears to have active flaring, making it an interesting subject for study.

For more detailed information about the flare activities observed during the Hinode mission, you can refer to the [XRT Flare Catalog](https://xrt.cfa.harvard.edu/flare_catalog/).


In [ ]:
# Define the time range of interest for solar observations.
time_range = a.Time("2021-05-21 18:51:00", "2021-05-22 01:59:54")

# Specify the instrument as 'xrt' to search for Hinode X-Ray Telescope data.
instrument = a.Instrument("xrt")


# This will return a catalog of available XRT data during the specified period.
xrt_files = Fido.search(time_range, instrument)

## Downloading Hinode XRT Data Using `Fido.fetch`

After successfully searching for the desired Hinode XRT data using Fido, the next step is to download the files. We use the `Fido.fetch` function for this purpose. This function takes the search results as input and downloads the data to a specified location on your local machine.

In the following code cell, we'll demonstrate how to use `Fido.fetch` to download the data we searched for in the previous step. We will also cover how to specify a download directory and handle the downloaded files.

In [ ]:
# Note: Depending on the amount of data and network speed, this process can take some time.
# The `progress` parameter in `Fido.fetch` controls the display of the download progress bar.
# Setting `progress=False` disables the progress bar, which can be useful for cleaner output,
# especially when running this in a script or automated pipeline. By default, we keep it True
# for this interactive session to monitor the download progress.

# This will download the files to the default SunPy data directory or a specified path.
# Replace `xrt_downloaded_files` with your search results variable if different.
xrt_files_results = Fido.fetch(xrt_files, progress=True)

# If you wish to specify a different download directory, you can do so by adding the `path` parameter:
# xrt_files_results = Fido.fetch(xrt_downloaded_files, path='/desired/download/directory/', progress=True)

[FITS]:(https://fits.gsfc.nasa.gov/)

XRT follows with standard Flexiable Image Transport System ([FITS]) file naming conventions. 

<a id="inspecting-the-downloaded-data"></a>
## Inspecting the Downloaded Data


Once the download is complete, it's important to inspect the data to ensure that what we have collected is suitable for our analysis. This step involves checking the basic information about the downloaded files, such as their names, file sizes, and data types. This information can help us verify that the download was successful and that we have the correct data for our project.

We will use Python's built-in functions and some SunPy utilities to print out basic information about each file. This includes:
- The file name: To understand the naming convention and organize the files.
- The file size: To get an idea of the data volume and ensure the files are not corrupted (e.g., unusually small).
- The data type or format: To confirm we have the right type of data for our processing and analysis.

In the next code cell, we will demonstrate practical methods to retrieve and display these details using Python's standard utilities along with SunPy-specific functions.

In [ ]:
# Number of files downloaded
num_files = len(xrt_files_results)
print(f"Number of files downloaded: {num_files}")

# Displaying the first few file names
print("\nSample file names:")
for file in xrt_files_results[:5]:
    print(file)

# Basic data check with SunPy map
# Creating a SunPy Map object for a randomly chosen file (100th file in this example)
sample_file_index = 84  # Index of the sample file
sample_data = sunpy.map.Map(xrt_files_results[sample_file_index])
print("\nSample data info for file number 100:")
print(sample_data)

# Visualizing the selected file #100
plt.figure(figsize=(10, 8))
sample_data.peek()  # 'peek' provides a quick look at the data with an automatically generated plot
plt.show()

Having successfully collected the Hinode-XRT data, we now have several exciting options at our disposal. We can analyze the FITS files to understand the solar phenomena, visualize the observations in image form, or even create a dynamic movie showcasing a solar event. However, we can take further step towards cleaning up some of the data we have downloaded.

<a id="exploring-functions-to-enhance-data-quality"></a>
## Exploring Functions to Enhance Data Quality

In the upcoming sections, we'll introduce some custom functions designed for filtering and refining the downloaded data. These functions are crafted to be user-friendly and straightforward, providing a foundation that can be built upon for more complex data filtering requirements. This step is optional but highly recommended for a clearner understanding of the data we've collected.

The following cells contain optional but beneficial steps to get a better sense of the quality and characteristics of the data we've downloaded. By exploring these functions, you'll gain insights into simple yet effective methods to filter and clean your data, laying the groundwork for more advanced and tailored analysis.


<a id="filtering-fits-files-through-selected-filter"></a>
### Filtering FITS Files Through Selected Filter


It is difficult to know what type of data is contained in XRT FITS files. The titles do not give enough information. I've written a function that will read the header information of every FITS file using `sunpy.map`, find the unique filters in the data, and finally clean out the data for a filter of interest. Simple run the code and fill in the prompted when shown. 

In [ ]:
def normalize_string(s):
    """
    Normalize a string for comparison.
    Converts to lowercase, replaces hyphens and underscores with spaces, and removes 'open'.
    """
    return (
        s.lower()
        .replace("open", "")
        .strip("- ")
        .replace("_", " ")
        .replace("-", " ")
        .strip()
    )


def filter_fits_files_by_XRT_filter(fits_files):
    """
    Filters FITS files by XRT filter criteria, accommodating flexible user input formats, enhancing readability,
    and ensuring valid input.

    Parameters:
    - fits_files: List of FITS file paths.

    Returns:
    - filtered_files: List of file paths that match the user-selected XRT channel filter.

    Raises:
    - ValueError: If the user input does not match any available filter.
    """
    measurement_info = {}

    for file_path in fits_files:
        try:
            from sunpy.map import Map

            sunpy_map = Map(file_path)
            measurement = sunpy_map.measurement
            # Process measurement for consistent comparison and improved readability
            processed_measurement = (
                measurement.replace("Open", "").strip("- ").replace("  ", " ")
            )
            normalized_measurement = normalize_string(processed_measurement)
            if normalized_measurement in measurement_info:
                measurement_info[normalized_measurement] += 1
            else:
                measurement_info[normalized_measurement] = 1
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    # Format the output for better readability
    filters_output = "\n".join(
        [
            f"{key.title().strip()}: {value} files"
            for key, value in measurement_info.items()
        ]
    )
    print(f"The files have the following filters and counts:\n{filters_output}")
    filter_choice = input("Please select an XRT channel filter of interest: ")
    # Normalize user input for comparison
    normalized_filter_choice = normalize_string(filter_choice)

    # Validate user input
    if normalized_filter_choice not in measurement_info:
        raise ValueError(
            "Invalid filter choice. Please enter a valid XRT channel filter from the list provided."
        )

    filtered_files = []

    for file_path in fits_files:
        try:
            sunpy_map = Map(file_path)
            measurement = sunpy_map.measurement
            # Process measurement for comparison
            processed_measurement = (
                measurement.replace("Open", "").strip("- ").replace("  ", " ")
            )
            normalized_measurement = normalize_string(processed_measurement)

            # Check if the normalized measurement matches the normalized filter criteria
            if normalized_filter_choice in normalized_measurement:
                filtered_files.append(file_path)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
    name_of_filter = filter_choice.title().strip()
    print(f"\nFilter choice: {name_of_filter}")
    print(
        f"You have {len(filtered_files)} FITS files that match the '{name_of_filter}' filter."
    )
    print("Be sure to store this data as a new variable.")

    return filtered_files

In [ ]:
# Assuming xrt_files_results is a list of FITS file paths
xrt_Al_poly_Obs_FITs = filter_fits_files_by_XRT_filter(xrt_files_results)

In [ ]:
# Enhanced Information Display of FITS Data
print("Data Overview:\n")

# Displaying the total number of FITS files
print(f"Total number of FITS files: {len(xrt_Al_poly_Obs_FITs)}\n")

# Displaying the first few file names to give a sample of the dataset
print("Sample file names:")
for file in xrt_Al_poly_Obs_FITs[:5]:
    print(f"  - {file}")

# Providing the first and last file as a quick data date and time range overview
print(f"\nFirst file in the dataset: {xrt_Al_poly_Obs_FITs[0]}")
print(f"Last file in the dataset: {xrt_Al_poly_Obs_FITs[-1]}")

<a id="filtering-fits-files-through-pixel-size"></a>
### Filtering FITS Files Through Pixel Size

Filtering Hinode XRT FITS files by pixel dimensions, measured in units of pixels (`pix`), is a practical approach to ensure data uniformity. Essential for accurate and efficient scientific analysis, visualization, and data management. We refer to these **dimensions** within the context of `sunpy.map.Map`.

XRT images typically have dimensions of `[384x384, 512x512, or 1024x1024]`, although any specific area of the CCD can be read out. For more information, please refer to the [SolarSoft XRT Analysis Guide](https://xrt.cfa.harvard.edu/resources/documents/XAG/XAG.pdf).


The `filter_fits_files_by_XRT_dimensions` function takes a list of FITS files and returns a new list filtered by a selected XRT dimension. Upon execution, you will be prompted with the available dimensions within your list of FITS files and asked to select your preferred option. Following your selection, a new set of filtered FITS files will be generated for your analysis. Simply run the function below and proceed to the subsequent cell:

In [ ]:
def filter_fits_files_by_XRT_dimensions(fits_files):
    """
    Filters FITS files by their dimensions, allowing for flexible input formatting.
    Provides up to three attempts for input, with error handling for invalid formats.

    Parameters:
    - fits_files (list): List of paths to the FITS files.

    Returns:
    - A list of file paths that match the user-selected dimensions.
    """
    dimension_counts = {}
    
    # Determine the unique dimensions of all FITS files and count them
    for file_path in fits_files:
        xrt_map = sunpy.map.Map(file_path)
        # Construct a tuple for dimensions to facilitate comparison
        dimensions_tuple = (xrt_map.dimensions.x.value, xrt_map.dimensions.y.value)
        dimensions_str = f"{dimensions_tuple[0]:.0f}, {dimensions_tuple[1]:.0f}"
        dimension_counts[dimensions_str] = dimension_counts.get(dimensions_str, 0) + 1
    
    print("Unique dimensions (in pixels) found in the dataset and their counts:\n x:pix , y:pix")
    for dimensions, count in sorted(dimension_counts.items()):
        print(f"{dimensions}: {count} files")
    
    attempts = 0
    while attempts < 3:
        chosen_dimensions_str = input("\nPlease enter the dimensions of interest (e.g., '384, 384'): ")
        try:
            x_dim, y_dim = [int(dim.strip()) for dim in chosen_dimensions_str.split(',')]
            chosen_dimensions_tuple = (x_dim, y_dim)
            chosen_dimensions_str = f"{x_dim}, {y_dim}"  # Reformatted string for comparison
        except ValueError:
            print("Invalid format. Please enter dimensions in the format 'X, Y'.")
            attempts += 1
            continue
        
        # Check if the processed input matches any known dimensions
        if chosen_dimensions_str in dimension_counts:
            break  # Valid dimension found; proceed with filtering
        else:
            print(f"Error: '{chosen_dimensions_str}' is not a recognized dimension.")
            attempts += 1
    
    if attempts == 3:
        print(f"\nMaximum attempts reached. Please rerun and try again. \nIf further issues persist, please email xrtpy@cfa.harvard.edu.")
        return []

    filtered_files = []
    for file_path in fits_files:
        xrt_map = sunpy.map.Map(file_path)
        if (xrt_map.dimensions.x.value, xrt_map.dimensions.y.value) == chosen_dimensions_tuple:
            filtered_files.append(file_path)
    
    print(f"\nFound {len(filtered_files)} files matching the dimensions {chosen_dimensions_str}.")
    return filtered_files


In [ ]:
# For this example, I will continue to use xrt_Al_poly_Obs_FITs. The function will prompt you to select from
# available dimensions and return a new list containing only the files that match the selected dimensions.

# Now, call the function with your list of FITS file paths:
xrt_Al_poly_384_Obs_FITs = filter_fits_files_by_XRT_dimensions(xrt_Al_poly_Obs_FITs)


<a id="Navigating_FITS_Images"></a>
## Navigating FITS Images 

Now, we have the capability to review images within our curated list of filtered FITS files. This enhanced navigation functionality not only facilitates sequential browsing but also introduces the ability to directly access a specific image within the dataset. Such a feature is invaluable for evaluating your dataset in detail, allowing you to identify and exclude FITS files that may not be relevant to your analysis or contain poor-quality images captured by the XRT.

Using the `view_fits_images` function, you can move forward or backward through the image dataset or jump to an Image directly navigate to a specific image by entering its number in the dataset. This feature is particularly useful when dealing with large datasets, as it enables quick access to images of interest without the need to sequentially skim through potentially hundreds of files. Whether you're looking for a particular phase of a solar event or need to examine the quality of specific images, this tool enhances your workflow by making data access more efficient and user-friendly.

In [ ]:

def view_fits_images(fits_files):
    """
    Displays FITS images one at a time with navigation buttons and an option to jump to a specific image.
    
    Parameters:
    - fits_files (list): List of paths to the FITS files.
    """
    
    current_index = [0]  # Use a list to allow modifications from inner functions
    
    def show_image(index=None):
        """Displays the image at the current index and clears the previous output."""
        if index is not None:
            # Safely update the current index based on user input
            current_index[0] = max(0, min(index, len(fits_files) - 1))
        
        clear_output(wait=True)  # Clear the previous image and controls
        display(widgets.HBox([prev_button, next_button, jump_input, jump_button]))  # Redisplay the controls
        
        fits_path = fits_files[current_index[0]]
        sunpy_map = sunpy.map.Map(fits_path)
        
        # Display FITS file name and image counter
        print(f"File: {fits_path.split('/')[-1]} (Image {current_index[0] + 1} of {len(fits_files)})")
        
        plt.figure(figsize=(6, 6))
        sunpy_map.plot()
        plt.show()
        
        update_buttons_status()
    
    def go_next(btn):
        """Go to the next image."""
        if current_index[0] < len(fits_files) - 1:
            current_index[0] += 1
            show_image()
    
    def go_prev(btn):
        """Go to the previous image."""
        if current_index[0] > 0:
            current_index[0] -= 1
            show_image()
    
    def jump_to_image(btn):
        """Jump to the image number entered by the user."""
        try:
            index = int(jump_input.value) - 1  # Convert to 0-based index
            show_image(index)
        except ValueError:
            print("Please enter a valid image number.")
    
    def update_buttons_status():
        """Updates the status of next/prev buttons based on the current index."""
        next_button.disabled = current_index[0] >= len(fits_files) - 1
        prev_button.disabled = current_index[0] <= 0
    
    # Create navigation buttons
    next_button = widgets.Button(description="Next")
    prev_button = widgets.Button(description="Previous")
    next_button.on_click(go_next)
    prev_button.on_click(go_prev)
    
    # Create jump to image widgets
    jump_input = widgets.Text(description="Jump to Image:", placeholder="Enter image number")
    jump_button = widgets.Button(description="Go")
    jump_button.on_click(jump_to_image)
    
    # Initially display buttons and the first image
    display(widgets.HBox([prev_button, next_button, jump_input, jump_button]))
    show_image()



#### Simple and Intuitive Use

1. **Navigation**: Click on the "Next" or "Previous" buttons to move through your dataset one image at a time.
2. **Direct Jump**: In the "Jump to Image" field, enter the image number you wish to view and click "Go".

The image number corresponds to its position in your list of FITS files, providing a straightforward way to access any image data directly.


In [ ]:
# We're going to use our filtered FITS data set: xrt_Al_poly_384_Obs_FITs
view_fits_images(xrt_Al_poly_384_Obs_FITs)

<a id="Removing-unwanted-FITS-Files"></a>
### Removing Unwanted FITS Files

After reviewing the FITS images using the `view_fits_images` function, you may identify some images that you wish to remove from your dataset. This could be due to poor quality, irrelevance to your study, or any other reason. Below we provide a straightforward method to facilitate this, allowing you to easily remove these unwanted files from your dataset.

#### Strategy for Removal

1. **Identification**: Note down either the index number(s) or the filename(s) of the images you want to remove.
2. **Compilation**: Create a Python [list](https://docs.python.org/3/tutorial/datastructures.html) with these indices or filenames.
3. **Execution**: Use a provided function to remove these files from your dataset list.

#### Implementation

Depending on your preference for using image numbers or filenames for identification, you can use one of the two provided functions.

#### Using Image Numbers

If using index numbers to identify images, use the following function:

In [ ]:
def remove_fits_by_index(original_list, indices_to_remove):
    """
    Removes FITS files from the dataset based on their index numbers.
    
    Parameters:
    - original_list (list): The original list of FITS file paths.
    - indices_to_remove (list): A list of index numbers representing the FITS files to remove.
    
    Returns:
    - A new list with the specified FITS files removed.
    """
    return [item for idx, item in enumerate(original_list) if idx not in indices_to_remove]

In [ ]:
# Example list of indices to remove
indices_to_remove = [2, 5, 7]  # Based on the order in the viewing function

# New updated list
updated_index_xrt_Al_poly_384_Obs = remove_fits_by_index(xrt_Al_poly_384_Obs_FITs, indices_to_remove)

#### Using Filenames
If you prefer noting down filenames:

In [ ]:
def remove_fits_by_filename(original_list, filenames_to_remove):
    """
    Removes FITS files from the dataset based on their filenames.
    
    Parameters:
    - original_list (list): The original list of FITS file paths.
    - filenames_to_remove (list): A list of filenames representing the FITS files to remove.
    
    Returns:
    - A new list with the specified FITS files removed.
    """
    return [item for item in original_list if item.split('/')[-1] not in filenames_to_remove]


In [ ]:
# Example list of filenames to remove
filenames_to_remove = ['l1_xrt20210521_185300_6.fits', 'l1_xrt20210521_185720_2.fits', 'l1_xrt20210521_190120_3.fits']  # Example filenames

# New updated list
updated_FITS_xrt_Al_poly_384_Obs = remove_fits_by_filename(xrt_Al_poly_384_Obs_FITs, filenames_to_remove)

<a id="Creating-a-Movie-from-XRT-FITS-Data"></a>
## Visualizing Solar Dynamics: Creating a Movie from XRT FITS Data

Creating a movie from FITS files is an excellent way to visualize data, particularly for dynamic solar phenomena observed by the Hinode XRT. We can accomplish this using SunPy to handle FITS files and matplotlib, along with imageio, to create the animation.

The `create_solar_movie_from_FITS` function generates a movie (MP4 format) from the FITS files you provide. If you're interested in customizing this process, you can review and modify the function as needed. To proceed with creating a movie using the default settings, run the cell containing the function below. Then, move to the following cell for instructions on how to use this function in your workflow.

In [ ]:
def create_solar_movie_from_FITS(
    fits_files, output_file="solar_movie.mp4", fps=5, processing=True
):
    """
    Creates a movie from a sequence of FITS files.

    Parameters:
    - fits_files (list): List of paths to the FITS files.
    - output_file (str): Path where the output movie will be saved.
    - fps (int): Frames per second for the output movie.
    - processing (bool): If True, prints processing messages. Default is True.
    """
    # Create a temporary directory to store the frames
    frames_dir = tempfile.mkdtemp()
    frames = []

    if processing:
        print(f"Starting to process {len(fits_files)} FITS files.\n")

    for i, file_path in enumerate(fits_files):
        if processing:
            print(f"Processing file {i+1}/{len(fits_files)}: {file_path}")
        # Load the FITS file as a SunPy Map
        xrt_map = sunpy.map.Map(file_path)

        # Plotting the SunPy Map
        fig = plt.figure(figsize=(8, 8))
        ax = plt.subplot(projection=xrt_map)
        xrt_map.plot(axes=ax)

        # Adding title and labels
        plt.title(
            f'XRT {xrt_map.measurement} {xrt_map.date.strftime("%Y-%m-%d %H:%M:%S")}'
        )
        ax.set_xlabel("Helioprojective Longitude (Solar-X)")
        ax.set_ylabel("Helioprojective Latitude (Solar-Y)")
        plt.tight_layout()

        # Saving the frame
        frame_path = os.path.join(frames_dir, f"frame_{i:04d}.png")
        plt.savefig(frame_path)
        plt.close(fig)
        frames.append(frame_path)
        if processing:
            print(f"Saved frame {i+1}/{len(fits_files)}")

    if processing:
        print("\nStarting to compile the movie.\n")

    # Compile the movie from saved frames
    with imageio.get_writer(output_file, fps=fps) as writer:
        for frame_path in frames:
            image = imageio.imread(frame_path)
            writer.append_data(image)

    # Cleanup: Remove temporary frames and directory
    for frame_path in frames:
        os.remove(frame_path)
    os.rmdir(frames_dir)

    if processing:
        print(
            f"\nMovie created: {output_file}\nAll done!\nMake sure to download the movie to your local machine to see it."
        )

In [ ]:
# Specify the output file name for the movie
solar_movie_example = "solar_movie_example.mp4"

# Create the movie from the list of FITS files
# Using our Al-Poly by 384X384 filtered FITs files
create_solar_movie_from_FITS(
    updated_FITS_xrt_Al_poly_384_Obs, 
    output_file=solar_movie_example, 
    fps=20, 
    processing=True
)

In [ ]:
# from IPython.display import Video

# Display the created movie within the notebook. Might to add "embed=True" to enable videos directly into a Jupyter Notebook
Video(solar_movie_example) 